### Abgleich verschiedener Vokabularien

In [2]:
import csv
import json
import nltk

In [3]:
#Import von HowTo (ht) - manuell ergänzte Version mit 17 Tags

with open("HowTo_ergänzt.json", encoding = "utf-8") as datei:
    ht = json.load(datei)

print(ht)

{'Datenmanagement': {'description': 'Datenmanagement', 'count': '4'}, 'Prosopographic data': {'description': 'Prosopographic data', 'count': '4'}, 'Semantic web': {'description': 'Semantic web', 'count': '4'}, 'Lexicography': {'description': 'The theory and practice of compiling dictionaries.', 'count': '1'}, 'Linguistics': {'description': 'Linguistics', 'count': '1'}, 'UI design': {'description': 'UI (user interface) design', 'count': '1'}, 'TEI': {'description': 'TEI', 'count': '1'}, 'Ontology': {'description': 'An ontology represents entities, data and ideas through the definition of categories, their properties, their interdependencies and relations, that show how the concepts, data and entities are related to a particular subject area, i.e. discourse. An ontology limits the complexity of a subject area and groups data into information and knowledge. When a controlled vocabulary is used to talk about the information and knowledge that are represented in an ontology, communication i

In [4]:
#Import von DARIAH-Campus (dc)

with open("DARIAH-Campus_Topics.json", encoding = "utf-8") as datei:
    dc = json.load(datei)

print(dc)

{'Research infrastructures': '35', 'Data management': '26', 'DH': '22', 'Open science': '21', 'eHeritage': '20', 'Training and education': '19', 'Digital Archives': '14', 'Open access': '13', 'Scholarly editions': '10', 'Scholarly practice': '8', 'Data visualisation': '7', 'Citizen science': '6', 'EOSC': '6', 'Multimodality': '6', 'Computational Imaging': '4', 'Data modeling': '4', 'Design Thinking': '4', 'Open education': '4', 'Project management': '4', 'Repositories & Collections': '4', 'Sustainability': '4', 'TEI': '4', 'XML': '4', 'Feminism': '3', 'Lexicography': '3', 'Metadata': '3', 'Scholarly publishing': '3', 'Source Criticism': '3', 'Spatial humanities': '3', 'Augmented reality': '2', 'Big data': '2', 'Controlled Vocabularies': '2', 'History of Technology': '2', 'Ontologies': '2', 'Semantic Web': '2', 'Software development': '2', 'Sound studies': '2', 'Editing tools': '1', 'Game Studies': '1', 'Geotagging': '1', 'Information Architecture': '1', 'Literacies': '1', 'Machine Lear

In [5]:
dc_ht = [] #Menge an genauen Überschneidungen (Groß-/Kleinschreibung ignoriert)

for dc_tag in dc.keys():
    for ht_tag in ht.keys():
        if dc_tag.lower() == ht_tag.lower():
            dc_ht.append(dc_tag)

print(dc_ht)
print(len(dc_ht))

['Open access', 'TEI', 'XML', 'Lexicography', 'Semantic Web', 'SKOS']
6


In [6]:
#Test: Levensthein-Distanz nutzen, um weitere ähnliche Fälle zu finden (Groß-/Kleinschreibung ignoriert)

for dc_tag in dc.keys():
    for ht_tag in ht.keys():
        distance = nltk.edit_distance(dc_tag.lower(), ht_tag.lower()) #Berechnung von Levensthein-Distanz
        if distance > 1 and distance < 3: #nicht identisch, aber auch nicht zu weit entfernt
            print(dc_tag, ht_tag)

Data management Datenmanagement
DH RDF
XSLT XML


### Zwischenfazit: DARIAH-Campus und ACDH-CH HowTo

- DARIAH-Campus: 49 Konzepte | ACDH-CH HowTo: 17 Konzepte
- hierunter 6 exakte Übereinstimmungen (Groß-/Kleinschreibung ignoriert):
    - "Open access"
    - "TEI"
    - "XML"
    - "Lexicography"
    - "Semantic web"
    - "SKOS"
- plus 1 Fall durch Levensthein-Distanz identifiziert: Data management/Datenmanagement

--> **Fazit**: 42 Konzepte nur in DARIAH-Campus -  7 Überschneidungen - 10  Konzepte nur im ACDH-CH HowTo

### Weitere Beobachtungen (CMS des HowTo/Auskunft von Susanne Zhanial)

- Anpassung des HowTo-Vokabulars an DARIAH-Campus geplant (Übernahme von deren Tags bzw. bei Bedarf Ergänzen von HowTo-zentralen Tags (z.B. "UI Design") in DARIAH-Campus)
- Zeitplan der Anpassung: voraussichtlich Ende Sommer/Herbst

### Systematischer Vergleich mehrerer Vokabularien - hier aktuell: DARIAH-Campus, HowTo, ACDH-Website 

In [26]:
#Tags der ACDH-CH-Website laden - 59 Tags

acdh = []

with open("Tags_Website.csv", encoding = "utf-8") as datei:
    for row in datei:
        element = row.strip().split(";")[0]
        if element == "\ufeffTag" or element == "Summe":
            continue
        else:
            acdh.append(element)

print(len(acdh))
print(acdh)

59
['aesthetics of music', 'annotation', 'archaeology', 'austrian music studies', 'citizen science', 'codicology', 'corpora', 'corpus linguistics', 'critical editions of music', 'cultural heritage', 'cultural history', 'cultural studies', 'culture studies', 'dha conferences', 'digital editions', 'digital musicology', 'early modern studies', 'eLexicography', 'exploration space', 'for interns', 'georeferenced data', 'historical language', 'history', 'history of musicology', 'history of scholarship', 'iconography of music', 'language resources', 'language variation', 'linguistics', 'linked open data', 'literary studies', 'liturgy', 'longterm projects', 'media studies', 'medieval studies', 'MEI', 'metadata', 'morphology', 'music philology', 'music theatre', 'musical sources', 'open innovation', 'open science', 'paleography', 'Projects', 'prosopography', 'research infrastructures', 'semantic technologies', 'semantic web', 'SKOS', 'sociolinguistics', 'TEI', 'terminology', 'text technology', 

In [25]:
#ARCHE-Tags laden - 852 Tags

arche = []

with open("ARCHE.csv", encoding = "utf-8") as datei:
    for row in datei:
        element = row.strip().split(";")[0]
        if element == "{\ufeffhasSubject":
            continue
        else:
            arche.append(element)

print(len(arche))
print(arche)


852
['\ufeffhasSubject', 'pottery fragments', 'Geschichte', 'Grundbuch', 'historic administrative units', 'GIS', 'Brief', 'D.K.BRF', 'Durchschlag', 'Typoskript', 'aerial warfare', 'Second world war', 'Zwischenkriegszeit', 'multilingualism', 'English as a lingua franca', 'interaction', 'interculturality', 'Druck', 'Landgericht', 'District Captaincy', 'pottery', 'Zeitungsartikel', 'D.P.ZTA', 'Bezirksamt', 'part-of-speech tagging', 'lemmatization', 'Mappe', 'D.J.MAP', 'Originalmappe', 'Manuskript', 'Kopie', 'Aktenvermerk', 'D.J.AKV', 'Beschluss', 'D.J.BSS', 'County', 'Antrag', 'D.J.ANT', 'word tokenization', 'Typoskript mit handschriftlichen Überarbeitungen', 'Durchschlag mit handschriftlichen Überarbeitungen', 'Karl Kraus', 'S.F.KRA', 'C.S.PRI', 'Privatanklagedelikt', 'R.F.STR', 'Strafrecht', 'Buchgeschichte', 'Geschichte des Lesens', 'privater Buchbesitz', 'Durchschlag mit handschriftlichen Annotationen', 'Typoskript mit handschriftlichen Annotationen', 'D.J.URT', 'Pensionierung', 'Urte

In [28]:
#Erstellen eines Gesamtverzeichnisses aller Tags (jedes distinkte Tag einmal)

tags = []

for key in dc.keys():
    if key not in tags:
        tags.append(key)

for key in ht.keys():
    if key not in tags:
        tags.append(key)

for element in acdh:
    if element not in tags:
        tags.append(element)

for element in arche:
    if element not in tags:
        tags.append(element)

print(tags)

['Research infrastructures', 'Data management', 'DH', 'Open science', 'eHeritage', 'Training and education', 'Digital Archives', 'Open access', 'Scholarly editions', 'Scholarly practice', 'Data visualisation', 'Citizen science', 'EOSC', 'Multimodality', 'Computational Imaging', 'Data modeling', 'Design Thinking', 'Open education', 'Project management', 'Repositories & Collections', 'Sustainability', 'TEI', 'XML', 'Feminism', 'Lexicography', 'Metadata', 'Scholarly publishing', 'Source Criticism', 'Spatial humanities', 'Augmented reality', 'Big data', 'Controlled Vocabularies', 'History of Technology', 'Ontologies', 'Semantic Web', 'Software development', 'Sound studies', 'Editing tools', 'Game Studies', 'Geotagging', 'Information Architecture', 'Literacies', 'Machine Learning', 'Maker Culture', 'MOOC', 'SKOS', 'Speech technology', 'XPath', 'XSLT', 'Datenmanagement', 'Prosopographic data', 'Semantic web', 'Linguistics', 'UI design', 'Ontology', 'Linked Open Data', 'Controlled vocabulary'

In [30]:
#Systematischer Abgleich der maximalen Liste (s. vorheriger Schritt) mit allen Listen (Groß-/Kleinschreibung ignoriert)

vgl = {}
dc_lower = [x.lower() for x in dc.keys()]
ht_lower = [x.lower() for x in ht.keys()]
acdh_lower = [x.lower() for x in acdh]
arche_lower = [x.lower() for x in arche]

for element in tags:
    count_dc = 0
    count_ht = 0
    count_acdh = 0
    count_arche = 0
    if element.lower() in dc_lower:
        count_dc = 1
    if element.lower() in ht_lower:
        count_ht = 1
    if element.lower() in acdh_lower:
        count_acdh = 1
    if element.lower() in arche_lower:
        count_arche = 1
    sum = count_dc + count_ht + count_acdh + count_arche
    vgl[element.lower()] = {"count_dc": count_dc, "count_ht": count_ht, "count_acdh": count_acdh, "count_arche": count_arche, "sum": sum}

for key in vgl.keys():
    print(key, "dc:", vgl[key]["count_dc"], "ht:", vgl[key]["count_ht"], "acdh:", vgl[key]["count_acdh"], "arche:", vgl[key]["count_arche"], "sum:", vgl[key]["sum"])

research infrastructures dc: 1 ht: 0 acdh: 1 arche: 0 sum: 2
data management dc: 1 ht: 0 acdh: 0 arche: 0 sum: 1
dh dc: 1 ht: 0 acdh: 0 arche: 0 sum: 1
open science dc: 1 ht: 0 acdh: 1 arche: 0 sum: 2
eheritage dc: 1 ht: 0 acdh: 0 arche: 0 sum: 1
training and education dc: 1 ht: 0 acdh: 0 arche: 0 sum: 1
digital archives dc: 1 ht: 0 acdh: 0 arche: 0 sum: 1
open access dc: 1 ht: 1 acdh: 0 arche: 0 sum: 2
scholarly editions dc: 1 ht: 0 acdh: 0 arche: 0 sum: 1
scholarly practice dc: 1 ht: 0 acdh: 0 arche: 0 sum: 1
data visualisation dc: 1 ht: 0 acdh: 0 arche: 0 sum: 1
citizen science dc: 1 ht: 0 acdh: 1 arche: 0 sum: 2
eosc dc: 1 ht: 0 acdh: 0 arche: 0 sum: 1
multimodality dc: 1 ht: 0 acdh: 0 arche: 0 sum: 1
computational imaging dc: 1 ht: 0 acdh: 0 arche: 0 sum: 1
data modeling dc: 1 ht: 0 acdh: 0 arche: 0 sum: 1
design thinking dc: 1 ht: 0 acdh: 0 arche: 0 sum: 1
open education dc: 1 ht: 0 acdh: 0 arche: 0 sum: 1
project management dc: 1 ht: 0 acdh: 0 arche: 0 sum: 1
repositories & coll

In [32]:
#Ausgabe und Zählung jener Tags, die in mehr als einem Vokabular vorkommen

count_2 = 0
count_3 = 0
count_4 = 0

for key in vgl.keys():
    if vgl[key]["sum"] == 2: 
        print(key, "--- dc:", vgl[key]["count_dc"], "| ht:", vgl[key]["count_ht"], "| acdh:", vgl[key]["count_acdh"], "| arche:", vgl[key]["count_arche"], "| sum:", vgl[key]["sum"])
        count_2 = count_2 + 1
    elif vgl[key]["sum"] == 3:
        print(key, "--- dc:", vgl[key]["count_dc"], "| ht:", vgl[key]["count_ht"], "| acdh:", vgl[key]["count_acdh"], "| arche:", vgl[key]["count_arche"], "| sum:", vgl[key]["sum"])
        count_3 = count_3 + 1
    elif vgl[key]["sum"] == 4:
        print(key, "--- dc:", vgl[key]["count_dc"], "| ht:", vgl[key]["count_ht"], "| acdh:", vgl[key]["count_acdh"], "| arche:", vgl[key]["count_arche"], "| sum:", vgl[key]["sum"])
        count_4 = count_4 + 1

print()
print("-->", count_2, "wörtliche Überschneidungen zwischen zwei Vokabularien")
print("-->", count_3, "wörtliche Überschneidungen zwischen drei Vokabularien")
print("-->", count_4, "wörtliche Überschneidungen zwischen allen vier Vokabularien")

research infrastructures --- dc: 1 | ht: 0 | acdh: 1 | arche: 0 | sum: 2
open science --- dc: 1 | ht: 0 | acdh: 1 | arche: 0 | sum: 2
open access --- dc: 1 | ht: 1 | acdh: 0 | arche: 0 | sum: 2
citizen science --- dc: 1 | ht: 0 | acdh: 1 | arche: 0 | sum: 2
tei --- dc: 1 | ht: 1 | acdh: 1 | arche: 1 | sum: 4
xml --- dc: 1 | ht: 1 | acdh: 0 | arche: 1 | sum: 3
lexicography --- dc: 1 | ht: 1 | acdh: 0 | arche: 0 | sum: 2
metadata --- dc: 1 | ht: 0 | acdh: 1 | arche: 0 | sum: 2
semantic web --- dc: 1 | ht: 1 | acdh: 1 | arche: 0 | sum: 3
skos --- dc: 1 | ht: 1 | acdh: 1 | arche: 0 | sum: 3
linguistics --- dc: 0 | ht: 1 | acdh: 1 | arche: 0 | sum: 2
ontology --- dc: 0 | ht: 1 | acdh: 0 | arche: 1 | sum: 2
linked open data --- dc: 0 | ht: 1 | acdh: 1 | arche: 0 | sum: 2
language variation --- dc: 0 | ht: 0 | acdh: 1 | arche: 1 | sum: 2

--> 10 wörtliche Überschneidungen zwischen zwei Vokabularien
--> 3 wörtliche Überschneidungen zwischen drei Vokabularien
--> 1 wörtliche Überschneidungen zw

In [33]:
#Speicherung des Vergleichs als csv

with open("Vokabularienvergleich.csv", "w", newline = "", encoding = "utf-8") as datei:
    writer = csv.writer(datei, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["Tag", "DARIAH-Campus", "HowTo", "ACDH-Website", "Summe"])
    for key in vgl.keys():
        writer.writerow(
            [key, vgl[key]["count_dc"], vgl[key]["count_ht"], vgl[key]["count_acdh"], vgl[key]["sum"]])

#Speicherung als Dictionary

with open("Vokabularienvergleich.json", "w", encoding = "utf-8") as datei:
    json.dump(vgl, datei, ensure_ascii = False)